In [2]:
from openparticle import *

/Users/carterm.gustin/anaconda3/envs/phd/lib/python3.9/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


$H(0) = H_f + H_{fb} + H_Y + H_+ + X_\Delta$

In 1 + 1D, 

$H_f = \sum_\sigma \int_p \frac{m^2}{p^+}b_p^\dagger \rightarrow \frac{1}{2L}\sum_\sigma \sum_n \frac{m^2}{p^+(n)}b_p^\dagger$

$H_{fb} = \sum_\sigma \int_{p,k} \left(\frac{m^2}{p^+} +\frac{\mu^2}{k^+}\right)b_p^\dagger a_k^\dagger \rightarrow \frac{1}{(2L)^2}\sum_\sigma \sum_{n,m} \left(\frac{m^2}{p^+(n)} +\frac{\mu^2}{k^+(m)}\right)b_p^\dagger a_k^\dagger$

$H_Y = g\sum_{\sigma_1, \sigma_2} \int_{p_1, p_2, k}\theta(\Delta^2) \delta(p_1 - p_2 - k) [\bar{u}_2u_1 b_2^\dagger b_1 a_0^\dagger + \bar{u}_1 u_2 b_1^\dagger b_2 a_0]$
$\rightarrow g\sum_{\sigma_1, \sigma_2}\sum_{n_1, n_2, m} \frac{1}{(2L)^3}\theta^\Delta \delta_{n_1 - n_1 - m} m_q\sqrt{p^+(n_1)p^+(n_2)}\left(\frac{1}{p^+(n_1)} + \frac{1}{p^+(n_2)}\right)[b_2^\dagger b_1 a_0^\dagger + b_1^\dagger b_2 a_0]$


$H_+ = g^2 \sum_{\sigma_1, \sigma_2} \int_{p_1, p_2, k_1, k_2} \theta(\Delta^2) \delta(p_1 + k_1 - p_2 - k_2) \bar{u}_2\frac{\gamma^+}{2(p_1^+ + k_1^+)}u_1$

$\rightarrow g^2 \sum_{\sigma_1, \sigma_2} \sum_{n_1, n_2, m_1, m_2} \theta_1^\Delta \theta_2^\Delta \delta_{n_1 + m_1 - n_2 - m_2}2 \sqrt{p^+(n_1)p^+(n_2)}\delta_{\sigma_1, \sigma_2}b_{n_2}^\dagger b_{n_1} a_{m_2}^\dagger a_{m_1} $

$\bar{u}_{\sigma'}(p')u_\sigma(p) = m\sqrt{p^+ p^{\prime +}} \left(\frac{1}{p^+} + \frac{1}{p^{\prime +}} \right)$

$\bar{u}_{\sigma'}(p')\gamma^+ u_\sigma(p) = 2\sqrt{p^+ p^{\prime +}}\delta_{\sigma', \sigma}$

In [3]:
m, mu = 1, 0.01
L = 1e4
Lambda = 3

In [4]:
def pplus(k):
    return 2 * np.pi * k / L

In [47]:
H0f = ParticleOperator({})
for spin in [-1, 1]:
    for mode_n, n in enumerate(np.arange(0.5, Lambda, 1)):
        # hf = m**2 / pplus(n)
        hf = 1
        H0f += hf * ParticleOperator('b' + str(mode_n) + "^")
# H0f = 1/(2*L) * H0f

In [61]:
H0fb = ParticleOperator({})
for spin in [-1, 1]:
    for mode_n, n in enumerate(np.arange(0.5, Lambda, 1)):
        for mode_m, m in enumerate(np.arange(0, Lambda, 1)):
            # hf = m**2 / pplus(n)
            # hb = mu**2 / (pplus(m) + 0.001)
            hf = hb = 1
            H0fb += (hf + hb) * ParticleOperator('b' + str(mode_n) + "^ " + 'a' + str(mode_m) + "^")
# H0fb = 1/((2*L)**2) * H0fb

In [62]:
H0 = H0f + H0fb

In [63]:
Hy = ParticleOperator({})

for mode_n1, n1 in enumerate(np.arange(0.5, Lambda, 1)):
    for mode_n2, n2 in enumerate(np.arange(0.5, Lambda, 1)):
        for mode_m, m in enumerate(np.arange(0, Lambda, 1)):
            if n1 - n2 - m == 0:
                # hy = np.sqrt(pplus(n1) * pplus(n2)) * (1/pplus(n1) + 1/pplus(n2))
                hy = 1
                op_str_1 = 'b' + str(mode_n2) + '^ ' + 'b' + str(mode_n1) + ' a' + str(mode_m) + '^'
                op_str_2 = 'b' + str(mode_n) + '^ ' + 'b' + str(mode_n2) + ' a' + str(mode_m)
                Hy += hy * (
                    ParticleOperator(op_str_1)
                    + ParticleOperator(op_str_2)
                )

In [64]:
Hplus = ParticleOperator({})

for sigma1 in [-1, 1]:
    for sigma2 in [-1, 1]:
        for mode_n1, n1 in enumerate(np.arange(0.5, Lambda, 1)):
            for mode_n2, n2 in enumerate(np.arange(0.5, Lambda, 1)):
                for mode_m1, m1 in enumerate(np.arange(0, Lambda, 1)):
                    for mode_m2, m2 in enumerate(np.arange(0, Lambda, 1)):
                        if n1 + m1 == n2 + m2:
                            if sigma1 == sigma2:
                                # hp = 2 * np.sqrt(pplus(n1) * pplus(n2))
                                hp = 1
                                Hplus += hp * (
                                    ParticleOperator('b' + str(mode_n2) + "^ " + \
                                                     'b' + str(mode_n1) + \
                                                    ' a' + str(mode_m2) + "^ " + \
                                                     'a' + str(mode_m1))
                                )

In [67]:
H = H0 + Hy + Hplus
H

4 * b0^ a0^
4 * b0^ a1^
4 * b0^ a2^
4 * b1^ a0^
4 * b1^ a1^
4 * b1^ a2^
4 * b2^ a0^
4 * b2^ a1^
4 * b2^ a2^
2 * b0^
2 * b1^
2 * b2^
1 * b0^ b0 a0^
1 * b2^ b0 a0
1 * b0^ b1 a1^
1 * b2^ b0 a1
1 * b1^ b1 a0^
1 * b2^ b1 a0
1 * b0^ b2 a2^
1 * b2^ b0 a2
1 * b1^ b2 a1^
1 * b2^ b1 a1
1 * b2^ b2 a0^
1 * b2^ b2 a0
2 * b0^ b0 a0^ a0
2 * b0^ b0 a1^ a1
2 * b0^ b0 a2^ a2
2 * b1^ b0 a0^ a1
2 * b1^ b0 a1^ a2
2 * b2^ b0 a0^ a2
2 * b0^ b1 a1^ a0
2 * b0^ b1 a2^ a1
2 * b1^ b1 a0^ a0
2 * b1^ b1 a1^ a1
2 * b1^ b1 a2^ a2
2 * b2^ b1 a0^ a1
2 * b2^ b1 a1^ a2
2 * b0^ b2 a2^ a0
2 * b1^ b2 a1^ a0
2 * b1^ b2 a2^ a1
2 * b2^ b2 a0^ a0
2 * b2^ b2 a1^ a1
2 * b2^ b2 a2^ a2